In [1]:
# !pip install sqlalchemy
# !pip install pymysql

In [2]:
# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
import sys
sys.path.append('..')
from config import remote_gwsis_dbuser, remote_gwsis_dbpwd, remote_db_host, remote_db_port, remote_gwsis_dbname

# Import Pandas
import pandas as pd

In [3]:
# Cloud MySQL Database Connection on AWS
engine = create_engine(f"mysql://{remote_gwsis_dbuser}:{remote_gwsis_dbpwd}@{remote_db_host}:{remote_db_port}/{remote_gwsis_dbname}")

In [4]:
# Create a remote database engine connection
conn = engine.connect()

In [5]:
state_codes = pd.read_csv('geojson/state_code.csv')
state_codes.head()

,State/District,Postal Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [6]:
data = pd.read_csv('tunnel/Tunnel_Details_Cleaned.csv')

tunnel_summary = data[["Tunnel_Number",
                       "Tunnel_Name",
                       "State_Code",
                       "Route_Type",
                       "Latitude",
                       "Longitude",
                       "Year_Built",
                       "Year_Rehabilitated",
                       "Lanes",
                       "AADT",
                       "AADTT",
                       "Detour Length",
                       "Toll",
                       "NHS",
                       "STRAHNET",
                       "Class",
                       "Length",
                       "Under_Nav_Water",
                       "Number_Bores",
                       "Tunnel_Shape",
                       "Ground_Conditions",
                       "Condition"
                        ]]

tunnel_summary.head()

,Tunnel_Number,Tunnel_Name,State_Code,Route_Type,Latitude,Longitude,Year_Built,Year_Rehabilitated,Lanes,AADT,...,Toll,NHS,STRAHNET,Class,Length,Under_Nav_Water,Number_Bores,Tunnel_Shape,Ground_Conditions,Condition
0,53C1210,Hope Street Tunnel,California,City Street,34.0483,-118.2570,1972,0,4,9000,...,0,0,1,Minor Arterial,75,0,1,Oval,Soil,2
1,1113,12th St SW Tunnel at Southwest Fwy,District of Columbia,City Street,38.8828,-77.0281,1959,0,4,10500,...,0,1,0,Minor Arterial,295,0,1,Oval,Soil,2
2,IN2MARION000001,South Capitol Avenue under CSX RR and Other Items,Indiana,City Street,39.7629,86.1619,1888,1984,4,11322,...,0,0,0,Minor Arterial,353,0,1,Oval,Soil,13
3,IN2MARION000002,Illinois Street under CSX RR and Other Items,Indiana,City Street,39.7620,86.1601,1888,1984,4,16130,...,0,0,0,Minor Arterial,310,0,1,Oval,Soil,4
4,IN2MARION000003,Meridian Street under CSX RR and Other Items,Indiana,City Street,39.7620,86.1583,1888,1984,4,9442,...,0,0,0,Minor Arterial,276,0,1,Oval,Soil,15


In [7]:
tunnel_summary = data.merge(state_codes, left_on=["State_Code"], right_on=["State/District"])
tunnel_summary = tunnel_summary[["Tunnel_Number",
                                 "Tunnel_Name",
                                 "State_Code",
                                 "Route_Type",
                                 "Latitude",
                                 "Longitude",
                                 "Year_Built",
                                 "Year_Rehabilitated",
                                 "Lanes",
                                 "AADT",
                                 "AADTT",
                                 "Detour Length",
                                 "Toll",
                                 "NHS",
                                 "STRAHNET",
                                 "Class",
                                 "Length",
                                 "Under_Nav_Water",
                                 "Number_Bores",
                                 "Tunnel_Shape",
                                 "Ground_Conditions",
                                 "Condition",
                                 "Postal Code"
                            ]]
tunnel_summary = tunnel_summary.rename(columns={"Postal Code":"State_Abbreviation"})
tunnel_summary.head()

,Tunnel_Number,Tunnel_Name,State_Code,Route_Type,Latitude,Longitude,Year_Built,Year_Rehabilitated,Lanes,AADT,...,NHS,STRAHNET,Class,Length,Under_Nav_Water,Number_Bores,Tunnel_Shape,Ground_Conditions,Condition,State_Abbreviation
0,53C1210,Hope Street Tunnel,California,City Street,34.0483,-118.257,1972,0,4,9000,...,0,1,Minor Arterial,75,0,1,Oval,Soil,2,CA
1,53C0202,Spring Street Tunnel,California,City Street,33.8117,-118.144,1978,0,2,26288,...,0,0,Local,1096,0,2,Oval,Soil,8,CA
2,54 0915,Mathews Ranch Rd Tunnel,California,City Street,34.2460,-117.436,1969,0,1,122000,...,1,1,Local,282,0,1,Oval,Soil,0,CA
3,55 0840L,Disney Way West Bound Tunnel 1,California,City Street,33.8056,-117.906,2000,0,2,5000,...,0,0,Local,265,0,1,Oval,Soil,0,CA
4,55 0840R,Disney Way West Bound Tunnel 2,California,City Street,33.8056,-117.905,2000,0,2,5000,...,0,0,Local,213,0,1,Oval,Soil,1,CA


In [8]:
tunnel_summary.to_sql(name='tunnels', if_exists='replace', con=conn, chunksize=500, index=True)